In [1]:
import numpy as np
from pyens.models import Flywheel, OCV
from pyens.utilities import ivp


ModuleNotFoundError: No module named 'matplotlib'

In [2]:
ocv = [3.3, 3.5, 3.55, 3.6, 3.65, 3.68, 3.70, 3.8, 3.95, 4.0, 4.1]
soc = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

c1 = OCV(name='curve1', ocv=ocv, soc=soc)

NameError: name 'OCV' is not defined

In [ ]:
param={'R0': 0.6,
       'R1': 0.1/1000, #mohm
       'C1': 10000,
       'R2': 0.05,
       'C2': 40000,
       'CAP': 42.94}
